In [1]:
! pip install hyperopt~=0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.7 MB/s eta 0:00:00


In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import mlflow

In [18]:
# Get data
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Without MLflow

In [33]:
%%time
def objective(max_depth: float) -> dict:
    '''
    Tries one model
    Should return a metric to be minimized (lower = better)
    '''
    print(f"Trying for max_depth={max_depth}")
    
    # Train model
    model = RandomForestRegressor(n_estimators=1, max_depth=int(max_depth))
    model.fit(X_train, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    return {'loss': -mae, 'status': STATUS_OK}

argmin = fmin(
  fn=objective,
  space=hp.quniform('max_depth', low=1, high=300, q=1), # space of values to test
  algo=tpe.suggest,
  max_evals=5,
)

print("Best value found: ", argmin)

Trying for max_depth=60.0                            
Trying for max_depth=4.0                             
Trying for max_depth=188.0                                                    
Trying for max_depth=274.0                                                    
Trying for max_depth=41.0                                                     
100%|██████████| 5/5 [00:00<00:00, 83.18trial/s, best loss: -67.3963963963964]
Best value found:  {'max_depth': 60.0}
CPU times: user 62.4 ms, sys: 8.2 ms, total: 70.6 ms
Wall time: 64.9 ms


# With MLflow

In [32]:
%%time
def objective(max_depth: float) -> dict:
    with mlflow.start_run():
        mlflow.log_params({'max_depth': max_depth})
        
        # Train model
        model = RandomForestRegressor(n_estimators=1, max_depth=int(max_depth))
        model.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=mlflow.models.infer_signature(X_train, model.predict(X_train)),
            input_example=X_train,
        )
        
        # Evaluate model
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mlflow.log_metric("mean_absolute_error_test", mean_absolute_error(y_test, y_pred))
        return {'loss': -mae, 'status': STATUS_OK}

mlflow.set_tracking_uri('http://server:5000')
mlflow.set_experiment("hyperopt1")
argmin = fmin(
  fn=objective,
  space=hp.quniform('max_depth', low=1, high=300, q=1), # space of values to test
  algo=tpe.suggest,
  max_evals=5,
)

print("Best value found: ", argmin)

100%|██████████| 5/5 [00:09<00:00,  1.90s/trial, best loss: -66.17117117117117]
Best value found:  {'max_depth': 173.0}
CPU times: user 654 ms, sys: 36.1 ms, total: 690 ms
Wall time: 9.52 s


# With MLflow hierarquical runs

In [31]:
%%time
def objective(max_depth: float) -> dict:
    with mlflow.start_run(nested=True):
        mlflow.log_params({'max_depth': max_depth})
        
        # Train model
        model = RandomForestRegressor(n_estimators=1, max_depth=int(max_depth))
        model.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=mlflow.models.infer_signature(X_train, model.predict(X_train)),
            input_example=X_train,
        )
        
        # Evaluate model
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mlflow.log_metric("mean_absolute_error_test", mean_absolute_error(y_test, y_pred))
        return {'loss': -mae, 'status': STATUS_OK}

mlflow.set_tracking_uri('http://server:5000')
mlflow.set_experiment("hyperopt2")
with mlflow.start_run(description='Parent run for hyperopt over max_depth', nested=True):
    argmin = fmin(
      fn=objective,
      space=hp.quniform('max_depth', low=1, high=300, q=1), # space of values to test
      algo=tpe.suggest,
      max_evals=5,
    )
    
    print("Best value found: ", argmin)

100%|██████████| 5/5 [00:09<00:00,  1.92s/trial, best loss: -65.34234234234235] 
Best value found:  {'max_depth': 98.0}
CPU times: user 672 ms, sys: 45.3 ms, total: 717 ms
Wall time: 9.67 s
